In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import time
from tqdm import tqdm

In [14]:
def scrap_from_one_page(driver, page_n, clothing_type):
    shein_df = pd.DataFrame()

    for post_id in range(1, 121):
        try:
            photo = driver.find_element(By.XPATH, f'/html/body/div[1]/div[1]/div[2]/div[2]/section/div[1]/section[{post_id}]/div[1]/a/img')
            photo.screenshot(f'data/photos/{clothing_type}/{clothing_type}_{page_n}_{post_id}.png')
            link = driver.find_element(By.XPATH, f'/html/body/div[1]/div[1]/div[2]/div[2]/section/div[1]/section[{post_id}]/div[2]/div[1]/a').get_attribute('href')
            price = driver.find_element(By.XPATH, f'/html/body/div[1]/div[1]/div[2]/div[2]/section/div[1]/section[{post_id}]/div[2]/div[2]/section/div[1]/span').text
            title = driver.find_element(By.XPATH, f'/html/body/div[1]/div[1]/div[2]/div[2]/section/div[1]/section[{post_id}]/div[2]/div[1]/a').text
            tmp_df = pd.DataFrame({
                'link': [link],
                'price': [price],
                'title': [title],
                'photo_id': [f'{clothing_type}_{page_n}_{post_id}']
            })
            shein_df = pd.concat([shein_df, tmp_df], ignore_index=True)
        except Exception as e:
            print(e)
            print(page_n, clothing_type, post_id)
    return shein_df

In [15]:
def scrap_from_several_pages(driver, n_pages, clothing_type):
    all_pages_df = pd.DataFrame()
    for page_n in tqdm(range(1, n_pages+1)):
        one_page_df = scrap_from_one_page(driver, page_n, clothing_type)
        one_page_df['page_n'] = page_n
        all_pages_df = pd.concat([all_pages_df, one_page_df], ignore_index=True)
        try:
            driver.find_element(By.XPATH, '//*[@id="product-list-v2"]/div[2]/div[2]/section/div[2]/nav/span[6]').send_keys(Keys.ENTER)
        except Exception as e:
            print(e)
            print('Maximum number of pages is reached')
            break
        time.sleep(2)
    return all_pages_df

In [16]:
def scrap_for_all_clothing_types(webdriver_path, n_pages):
    
    all_clothing_types = pd.DataFrame()
    types = ['tops', 'dresses', 'bottoms', 'two_piece_outfit',
             'sweatshirts', 'outwear', 'activewear', 'denim',
             'beachwear', 'intimates', 'sleep_and_lounge',
             'costumes', 'wedding', 'maternity']
    for clothing_type in types:
        print(clothing_type)
        if clothing_type == 'tops':
            url = 'https://roe.shein.com/Women-Tops,-Blouses-Tee-c-1766.html?ici=roe_tab01navbar05menu01&scici=navbar_WomenHomePage~~tab01navbar05menu01~~5_1~~real_1766~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657646979980&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DTOPS%60oc%3D0%60ps%3Dtab01navbar05menu01%60jc%3Dreal_1766&srctype=category&userpath=category-CLOTHING-TOPS'
        elif clothing_type == 'dresses':
            url = 'https://roe.shein.com/Women-Dresses-c-1727.html?ici=roe_tab01navbar05menu02&scici=navbar_WomenHomePage~~tab01navbar05menu02~~5_2~~real_1727~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647404709&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DDRESSES%60oc%3D0%60ps%3Dtab01navbar05menu02%60jc%3Dreal_1727&srctype=category&userpath=category-CLOTHING-DRESSES'
        elif clothing_type == 'bottoms':
            url = 'https://roe.shein.com/Women-Bottoms-c-1767.html?ici=roe_tab01navbar05menu03&scici=navbar_WomenHomePage~~tab01navbar05menu03~~5_3~~real_1767~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647416958&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DBOTTOMS%60oc%3D0%60ps%3Dtab01navbar05menu03%60jc%3Dreal_1767&srctype=category&userpath=category-CLOTHING-BOTTOMS'
        elif clothing_type == 'two_piece_outfit':
            url = 'https://roe.shein.com/Women-Two-piece-Outfits-c-1780.html?ici=roe_tab01navbar05menu04&scici=navbar_WomenHomePage~~tab01navbar05menu04~~5_4~~real_1780~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647453442&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DTWO-PIECE%20OUTFITS%60oc%3D0%60ps%3Dtab01navbar05menu04%60jc%3Dreal_1780&srctype=category&userpath=category-CLOTHING-TWO-PIECE-OUTFITS'
        elif clothing_type == 'jumpsuits_and_bodysuits':
            url = 'https://roe.shein.com/Women-Jumpsuits-Bodysuits-c-3287.html?ici=roe_tab01navbar05menu05&scici=navbar_WomenHomePage~~tab01navbar05menu05~~5_5~~real_3287~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647494673&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DJUMPSUITS%20%26%20BODYSUITS%60oc%3D0%60ps%3Dtab01navbar05menu05%60jc%3Dreal_3287&srctype=category&userpath=category-CLOTHING-JUMPSUITS-BODYSUITS'
        elif clothing_type == 'sweatshirts':
            url = 'https://roe.shein.com/Women-Sweatshirts-c-1773.html?ici=roe_tab01navbar05menu06&scici=navbar_WomenHomePage~~tab01navbar05menu06~~5_6~~real_1773~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647537755&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DSWEATSHIRTS%60oc%3D0%60ps%3Dtab01navbar05menu06%60jc%3Dreal_1773&srctype=category&userpath=category-CLOTHING-SWEATSHIRTS'
        elif clothing_type == 'sweaters_and_cardigans':
            url = 'https://roe.shein.com/Women-Knitwear-c-2216.html?ici=roe_tab01navbar05menu07&scici=navbar_WomenHomePage~~tab01navbar05menu07~~5_7~~real_2216~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647626531&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DSWEATERS%20%26%20CARDIGANS%60oc%3D0%60ps%3Dtab01navbar05menu07%60jc%3Dreal_2216&srctype=category&userpath=category-CLOTHING-SWEATERS-CARDIGANS'
        elif clothing_type == 'outwear':
            url = 'https://roe.shein.com/Women-Outerwear-c-2037.html?ici=roe_tab01navbar05menu08&scici=navbar_WomenHomePage~~tab01navbar05menu08~~5_8~~real_2037~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647661763&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DOUTERWEAR%60oc%3D0%60ps%3Dtab01navbar05menu08%60jc%3Dreal_2037&srctype=category&userpath=category-CLOTHING-OUTERWEAR'
        elif clothing_type == 'suits_and_separates':
            url = 'https://roe.shein.com/Women-Suits-c-2036.html?ici=roe_tab01navbar05menu09&scici=navbar_WomenHomePage~~tab01navbar05menu09~~5_9~~real_2036~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647779193&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DSUITS%20%26%20SEPARATES%60oc%3D0%60ps%3Dtab01navbar05menu09%60jc%3Dreal_2036&srctype=category&userpath=category-CLOTHING-SUITS-SEPARATES'
        elif clothing_type == 'activewear':
            url = 'https://roe.shein.com/Sports-c-3195.html?ici=roe_tab01navbar05menu10&scici=navbar_WomenHomePage~~tab01navbar05menu10~~5_10~~real_3195~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647821675&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DACTIVEWEAR%60oc%3D0%60ps%3Dtab01navbar05menu10%60jc%3Dreal_3195&srctype=category&userpath=category-CLOTHING-ACTIVEWEAR'
        elif clothing_type == 'denim':
            url = 'https://roe.shein.com/Women-Denim-c-1930.html?ici=roe_tab01navbar05menu11&scici=navbar_WomenHomePage~~tab01navbar05menu11~~5_11~~real_1930~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647913128&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DDENIM%60oc%3D0%60ps%3Dtab01navbar05menu11%60jc%3Dreal_1930&srctype=category&userpath=category-CLOTHING-DENIM'
        elif clothing_type == 'beachwear':
            url = 'https://roe.shein.com/Women-Beachwear-c-2039.html?ici=roe_tab01navbar05menu12&scici=navbar_WomenHomePage~~tab01navbar05menu12~~5_12~~real_2039~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657647982860&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DBEACHWEAR%60oc%3D0%60ps%3Dtab01navbar05menu12%60jc%3Dreal_2039&srctype=category&userpath=category-CLOTHING-BEACHWEAR'
        elif clothing_type == 'intimates':
            url = 'https://roe.shein.com/Women-Intimates-c-3625.html?ici=roe_tab01navbar05menu13&scici=navbar_WomenHomePage~~tab01navbar05menu13~~5_13~~real_3625~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657648025969&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DINTIMATES%60oc%3D0%60ps%3Dtab01navbar05menu13%60jc%3Dreal_3625&srctype=category&userpath=category-CLOTHING-INTIMATES'
        elif clothing_type == 'sleep_and_lounge':
            url = 'https://roe.shein.com/Women-Sleep-Lounge-c-3626.html?ici=roe_tab01navbar05menu14&scici=navbar_WomenHomePage~~tab01navbar05menu14~~5_14~~real_3626~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657648059107&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DWOMEN%20SLEEP%20%26%20LOUNGE%60oc%3D0%60ps%3Dtab01navbar05menu14%60jc%3Dreal_3626&srctype=category&userpath=category-CLOTHING-WOMEN-SLEEP-LOUNGE'
        elif clothing_type == 'costumes':
            url = 'https://roe.shein.com/Cosplay-Costumes-c-3066.html?ici=roe_tab01navbar05menu15&scici=navbar_WomenHomePage~~tab01navbar05menu15~~5_15~~real_3066~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657648100021&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DCOSTUMES%60oc%3D0%60ps%3Dtab01navbar05menu15%60jc%3Dreal_3066&srctype=category&userpath=category-CLOTHING-COSTUMES'
        elif clothing_type == 'wedding':
            url = 'https://roe.shein.com/Weddings-Events-c-3088.html?ici=roe_tab01navbar05menu16&scici=navbar_WomenHomePage~~tab01navbar05menu16~~5_16~~real_3088~~~~0&src_module=topcat&src_tab_page_id=page_home1658045956619&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DWEDDING%60oc%3D0%60ps%3Dtab01navbar05menu16%60jc%3Dreal_3088&srctype=category&userpath=category-CLOTHING-WEDDING'
        elif clothing_type == 'maternity':
            url = 'https://roe.shein.com/Maternity-c-2985.html?ici=roe_tab01navbar05menu17&scici=navbar_WomenHomePage~~tab01navbar05menu17~~5_17~~real_2985~~~~0&src_module=topcat&src_tab_page_id=page_real_class1657648164385&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DMATERNITY%60oc%3D0%60ps%3Dtab01navbar05menu17%60jc%3Dreal_2985&srctype=category&userpath=category-CLOTHING-MATERNITY'


        s = Service(executable_path = webdriver_path)
        driver = webdriver.Chrome(service=s)
        driver.get(url)

        # closing pop-up window
        driver.find_element(By.XPATH, '/html/body/div[1]/div[5]/div[1]/div/i').send_keys(Keys.ENTER)
        driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div[1]/div/button[1]').send_keys(Keys.ENTER)
        
        all_pages_df = scrap_from_several_pages(driver, n_pages, clothing_type)
        all_pages_df['clothing_type'] = clothing_type
        all_clothing_types = pd.concat([all_clothing_types, all_pages_df], ignore_index=True)
        driver.close()
    return(all_clothing_types)

In [17]:
webdriver_path = '/home/hem/Desktop/scraping/Project/chromedriver'
all_clothing_types = scrap_for_all_clothing_types(webdriver_path, 10)

tops


100%|███████████████████████████████████████████| 10/10 [05:19<00:00, 31.96s/it]


dresses


100%|███████████████████████████████████████████| 10/10 [05:04<00:00, 30.45s/it]


bottoms


...

100%|███████████████████████████████████████████| 10/10 [02:38<00:00, 15.87s/it]


wedding


100%|███████████████████████████████████████████| 10/10 [04:52<00:00, 29.23s/it]


maternity


100%|███████████████████████████████████████████| 10/10 [04:55<00:00, 29.57s/it]


In [18]:
all_clothing_types.clothing_type.value_counts()

tops                1200
dresses             1200
bottoms             1200
two_piece_outfit    1200
sweatshirts         1200
outwear             1200
activewear          1200
denim               1200
beachwear           1200
intimates           1200
sleep_and_lounge    1200
wedding             1200
maternity           1200
costumes             528
Name: clothing_type, dtype: int64

In [19]:
all_clothing_types.to_csv('data/shein_data.csv', index=False)

In [20]:
all_clothing_types.iloc[6]['link']

'https://roe.shein.com/Ribbed-Knit-Asymmetrical-Tank-Top-p-10877015-cat-1779.html?src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3DTOPS%60oc%3D0%60ps%3Dtab01navbar05menu01%60jc%3Dreal_1766&src_module=topcat&src_tab_page_id=page_real_class1657646979980&attr_ids=&scici=navbar_WomenHomePage~~tab01navbar05menu01~~5_1~~real_1766~~~~0'

In [21]:
all_clothing_types.shape

(16128, 6)

In [23]:
all_clothing_types.tail()

,link,price,title,photo_id,page_n,clothing_type
16123,https://roe.shein.com/SHEIN-Maternity-Button-D...,"17,00€",SHEIN Maternity Button Detail Ruffle Hem Dress,maternity_10_116,10,maternity
16124,https://roe.shein.com/SHEIN-Maternity-Puff-Sle...,"18,00€",SHEIN Maternity Puff Sleeve Knot Front Shirred...,maternity_10_117,10,maternity
16125,https://roe.shein.com/SHEIN-Maternity-Square-N...,"17,00€",SHEIN Maternity Square Neck Puff Sleeve Belted...,maternity_10_118,10,maternity
16126,https://roe.shein.com/SHEIN-Maternity-Batwing-...,"14,99€",SHEIN Maternity Batwing Sleeve Dress,maternity_10_119,10,maternity
16127,https://roe.shein.com/SHEIN-Maternity-Contrast...,"7,00€",SHEIN Maternity Contrast Eyelash Lace Scallop ...,maternity_10_120,10,maternity
